In [14]:
import pandas as pd
data = pd.read_csv('output.csv')
data

,name,link
0,Dreamcatcher,/book/show/11570.Dreamcatcher
1,Carnal: Somewhere Over 40 Winks,/book/show/7673244-carnal
2,I Am Malala: The Story of the Girl Who Stood U...,/book/show/17851885-i-am-malala
3,Possession of My Heart (The Three Immortal Bla...,/book/show/23291161-possession-of-my-heart
4,"Living Dead in Dallas (Sookie Stackhouse, #2)",/book/show/110494.Living_Dead_in_Dallas
...,...,...
9118,The Weight of Ink,/book/show/30971715-the-weight-of-ink
9119,"A Rush of Wings (A Rush of Wings, #1)",/book/show/226730.A_Rush_of_Wings
9120,Blue Nights,/book/show/10252302-blue-nights
9121,"A Curse So Dark and Lonely (Cursebreakers, #1)",/book/show/43204703-a-curse-so-dark-and-lonely


In [15]:
for i in range(len(data['link'])):
    link = "https://www.goodreads.com" + data['link'][i]
    data['link'][i] = link
#data[data['link'] == '/book/show/1.Harry_Potter_and_the_Half_Blood_Prince']
(data[data['link'] == 'https://www.goodreads.com/book/show/1.Harry_Potter_and_the_Half_Blood_Prince'])

,name,link
283,Harry Potter and the Half-Blood Prince (Harry ...,https://www.goodreads.com/book/show/1.Harry_Po...


In [23]:
import pandas as pd
review_count = []
pages = []
publishers = []
result = pd.read_csv('test.csv')
for i in range(len(result['rating_count'])):
    if not result['author'].isna()[i]:
        review_count.append(result['rating_count'][i].split()[4])
        pages.append(str(result['num_of_page'][i]).split()[0])

        result['rating_count'][i] = result['rating_count'][i].split()[1]
    
        date_and_publisher = result['date_published'][i].split('by')
        date = date_and_publisher[0].split()
   
       
        try:
            date.remove('Published')
        except:
            pass
        result['date_published'][i] = " ".join(date)
        try:            
            publisher = date_and_publisher[1].strip()
            publishers.append(publisher)
        except:
            publishers.append('NaN')


    else:
        review_count.append('NaN')
        pages.append('NaN')
        publishers.append('NaN')

result.insert(8,column="review_count",value=review_count)
result['num_of_page'] = pd.DataFrame(pages)
result.insert(6,'publisher',publishers,True)


2021-11-17 02:15:42 [py.warnings] WARNING: C:\Users\phucl\AppData\Local\Temp/ipykernel_11424/745853250.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['rating_count'][i] = result['rating_count'][i].split()[1]

2021-11-17 02:15:42 [py.warnings] WARNING: C:\Users\phucl\AppData\Local\Temp/ipykernel_11424/745853250.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['date_published'][i] = " ".join(date)



AttributeError: 'float' object has no attribute 'split'

In [18]:
import pandas as pd
result = pd.read_csv('test.csv')
a = result['date_published'][83].split('by')
a[0].split()

ParserError: Error tokenizing data. C error: Expected 11 fields in line 901, saw 22
